In [ ]:
# Projeto de Machine Learning com o Brazilian E-Commerce Public Dataset by Olist
# Etapa 1: Importação de bibliotecas e carregamento dos dados

# Bibliotecas para manipulação, visualização e machine learning
import os
import zipfile
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, KFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report,
                             mean_squared_error, mean_absolute_error, r2_score, silhouette_score, davies_bouldin_score)
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.svm import SVC, SVR
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
import plotly.express as px
import plotly.graph_objects as go

# Extração dos dados do zip (caso ainda não tenha sido extraído)
if not os.path.exists('olist_data'):
    with zipfile.ZipFile('olist.zip', 'r') as zip_ref:
        zip_ref.extractall('olist_data')

# Listar arquivos extraídos
files = sorted(os.listdir('olist_data'))
csv_files = [f for f in files if f.endswith('.csv')]
print('Arquivos CSV disponíveis:', csv_files)

# Carregar os principais datasets
orders = pd.read_csv('olist_data/olist_orders_dataset.csv')
order_items = pd.read_csv('olist_data/olist_order_items_dataset.csv')
products = pd.read_csv('olist_data/olist_products_dataset.csv')
payments = pd.read_csv('olist_data/olist_order_payments_dataset.csv')
reviews = pd.read_csv('olist_data/olist_order_reviews_dataset.csv')
sellers = pd.read_csv('olist_data/olist_sellers_dataset.csv')
customers = pd.read_csv('olist_data/olist_customers_dataset.csv')
geolocation = pd.read_csv('olist_data/olist_geolocation_dataset.csv')
category_translation = pd.read_csv('olist_data/product_category_name_translation.csv')

# Visualizar as primeiras linhas do dataset central
orders.head()

# Projeto de Machine Learning: Olist E-Commerce Dataset

Este notebook apresenta um projeto completo de ciência de dados utilizando o Brazilian E-Commerce Public Dataset by Olist. O objetivo é realizar uma análise exploratória detalhada, propor e resolver dois problemas de negócio críticos para a operação da Olist, implementar diferentes algoritmos de machine learning, e extrair insights acionáveis para melhorar a operação do marketplace.

## Problemas

1. Problema Supervisionado: Previsão do Tempo de Entrega
Objetivo: Criar um modelo que prevê o tempo exato de entrega (em dias) para cada pedido específico, considerando:
- Distância real entre vendedor e cliente (dados de geolocalização)
- Histórico de entregas do vendedor
- Tamanho/peso do produto
- Condições sazonais e região
- Performance do transportador
Justificativa: Precisão no prazo de entrega é crucial para satisfação do cliente e eficiência operacional.
Valor de Negócio: 
- Redução de reclamações por atrasos
- Melhor gestão de expectativas do cliente
- Otimização da logística
- Possibilidade de oferecer frete expresso com maior precisão
- Redução de custos com compensações por atrasos

2. Problema Não Supervisionado: Detecção de Vendedores Problemáticos
Objetivo: Identificar automaticamente vendedores suspeitos usando técnicas de detecção de anomalias, analisando:
- Padrões anormais de preços
- Taxa de cancelamento e devolução
- Velocidade de crescimento suspeita nas vendas
- Concentração geográfica incomum de avaliações
- Padrões de texto suspeitos em avaliações
- Histórico de reclamações
Justificativa: Proteção da reputação do marketplace e garantia de qualidade dos vendedores.
Valor de Negócio:
- Redução de fraudes e prejuízos
- Melhoria na qualidade geral dos vendedores
- Aumento da confiança dos compradores
- Redução de custos com suporte

Vamos começar!

# 2. Análise Exploratória dos Dados (EDA)

Nesta seção, vamos explorar os principais datasets do projeto, analisando estatísticas descritivas, valores ausentes, distribuições e relações relevantes entre as tabelas. O objetivo é compreender o contexto dos dados e identificar potenciais variáveis para os problemas de negócio.

In [ ]:
# Visualizar informações gerais dos principais datasets
def resumo_dataset(df, nome):
    print(f'\nResumo do dataset: {nome}')
    display(df.info())
    display(df.describe(include='all'))
    print(f'Valores ausentes em {nome}:')
    display(df.isnull().sum())

resumo_dataset(orders, 'orders')
resumo_dataset(order_items, 'order_items')
resumo_dataset(products, 'products')
resumo_dataset(payments, 'payments')
resumo_dataset(reviews, 'reviews')
resumo_dataset(sellers, 'sellers')
resumo_dataset(customers, 'customers')
resumo_dataset(geolocation, 'geolocation')

# 3. Limpeza e Tratamento dos Dados

Nesta etapa, vamos tratar valores ausentes, inconsistências e preparar os dados para a modelagem. Isso inclui:
- Remoção ou imputação de valores nulos
- Conversão de tipos de dados
- Ajuste de colunas de datas
- Verificação de duplicidades
- Padronização de categorias

In [ ]:
# Tratamento de valores ausentes e tipos de dados
# Exemplo para o dataset de pedidos (orders)

# Converter colunas de datas
orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'])
orders['order_approved_at'] = pd.to_datetime(orders['order_approved_at'])
orders['order_delivered_carrier_date'] = pd.to_datetime(orders['order_delivered_carrier_date'])
orders['order_delivered_customer_date'] = pd.to_datetime(orders['order_delivered_customer_date'])
orders['order_estimated_delivery_date'] = pd.to_datetime(orders['order_estimated_delivery_date'])

# Verificar valores ausentes
print('Valores ausentes em orders:')
display(orders.isnull().sum())

# Exemplo de tratamento: remover linhas sem data de entrega real (para problemas supervisionados)
orders_clean = orders.dropna(subset=['order_delivered_customer_date'])

# Repetir processo para outros datasets conforme necessário
# Exemplo: preencher valores nulos em reviews
reviews['review_comment_message'] = reviews['review_comment_message'].fillna('Sem comentário')
reviews['review_comment_title'] = reviews['review_comment_title'].fillna('Sem título')

# Conferir duplicidades
print('Pedidos duplicados:', orders_clean.duplicated(subset=['order_id']).sum())
print('Clientes duplicados:', customers.duplicated(subset=['customer_id']).sum())

In [ ]:
# Visualização da quantidade de pedidos por estado do cliente
clientes_estados = customers.merge(orders, on='customer_id')
clientes_estados = clientes_estados['customer_state'].value_counts().reset_index()
clientes_estados.columns = ['Estado', 'Quantidade de Pedidos']

plt.figure(figsize=(10,5))
sns.barplot(data=clientes_estados, x='Estado', y='Quantidade de Pedidos', palette='viridis')
plt.title('Quantidade de Pedidos por Estado do Cliente')
plt.xlabel('Estado')
plt.ylabel('Pedidos')
plt.show()# Visualização de distribuições e relações iniciais dos dados
import matplotlib.pyplot as plt
import seaborn as sns

# Distribuição do status dos pedidos
plt.figure(figsize=(8,4))
sns.countplot(data=orders, x='order_status', order=orders['order_status'].value_counts().index)
plt.title('Distribuição dos Status dos Pedidos')
plt.xlabel('Status')
plt.ylabel('Quantidade')
plt.show()

# Distribuição das avaliações dos clientes
plt.figure(figsize=(8,4))
sns.countplot(data=reviews, x='review_score', order=sorted(reviews['review_score'].unique()))
plt.title('Distribuição das Avaliações dos Clientes')
plt.xlabel('Nota de Avaliação')
plt.ylabel('Quantidade')
plt.show()

# Distribuição dos tipos de pagamento
plt.figure(figsize=(8,4))
sns.countplot(data=payments, x='payment_type', order=payments['payment_type'].value_counts().index)
plt.title('Distribuição dos Tipos de Pagamento')
plt.xlabel('Tipo de Pagamento')
plt.ylabel('Quantidade')
plt.show()

# 4. Definição dos Problemas de Negócio

Nesta seção, serão definidos dois problemas de negócio relevantes para a Olist:

- **Problema supervisionado:** Previsão da nota de avaliação do cliente (regressão). O objetivo é prever a nota que um cliente dará ao pedido, com base em informações do pedido, produto, entrega e pagamento. Isso permite à Olist identificar fatores que impactam a satisfação do cliente e atuar preventivamente para melhorar a experiência.

- **Problema não supervisionado:** Segmentação de clientes por comportamento de compra (clustering). O objetivo é agrupar clientes com perfis de compra semelhantes, permitindo ações de marketing direcionadas, personalização de ofertas e melhor compreensão da base de clientes.

A seguir, detalharemos as variáveis utilizadas, justificativas e valor de negócio de cada abordagem.

## 4.1 Problema Supervisionado: Previsão da Nota de Avaliação do Cliente

**Justificativa:**
- A nota de avaliação é um indicador direto da satisfação do cliente.
- Antecipar avaliações baixas permite ações proativas para reduzir churn e melhorar a reputação.

**Variáveis sugeridas:**
- Prazo de entrega (real x estimado)
- Valor do pedido
- Tipo de pagamento
- Quantidade de itens
- Categoria do produto
- Região do cliente
- Status do pedido

**Valor de negócio:**
- Redução de avaliações negativas
- Melhoria da experiência do cliente
- Aumento da fidelização

## 4.2 Problema Não Supervisionado: Segmentação de Clientes por Comportamento de Compra

**Justificativa:**
- A Olist pode personalizar campanhas e ofertas para diferentes perfis de clientes.
- Permite identificar grupos de alto valor, clientes recorrentes, caçadores de promoções, etc.

**Variáveis sugeridas:**
- Frequência de compras
- Ticket médio
- Diversidade de categorias compradas
- Região
- Forma de pagamento

**Valor de negócio:**
- Aumento da efetividade de marketing
- Personalização de ofertas
- Melhoria do relacionamento com o cliente

Na próxima etapa, os dados serão preparados para a modelagem, incluindo seleção de variáveis, criação de features e junção das tabelas necessárias.

# 5. Preparação dos Dados para Modelagem

Nesta etapa, vamos preparar os dados para os dois problemas definidos:
- Seleção e criação de variáveis (features)
- Junção das tabelas necessárias
- Engenharia de atributos
- Padronização e encoding

Primeiro, será feita a preparação para o problema supervisionado (previsão da nota de avaliação do cliente), seguida da preparação para o problema não supervisionado (segmentação de clientes).

## 5.1 Preparação para o Problema Supervisionado

Vamos criar um dataset unificado contendo, para cada pedido avaliado:
- Informações do pedido (datas, status, região)
- Informações do pagamento
- Informações do produto principal
- Prazo de entrega real x estimado
- Nota de avaliação (target)

In [ ]:
# Preparação para o problema de previsão de tempo de entrega
# 1. Juntar as tabelas necessárias
df_sup = orders_clean.merge(order_items, on='order_id')
df_sup = df_sup.merge(payments, on='order_id', how='left')  # Adiciona informações de pagamento
df_sup = df_sup.merge(products, on='product_id', how='left')
df_sup = df_sup.merge(category_translation, left_on='product_category_name', right_on='product_category_name', how='left')  # Adiciona coluna em inglês
df_sup = df_sup.merge(sellers[['seller_id', 'seller_city', 'seller_state']], on='seller_id', how='left')
df_sup = df_sup.merge(customers[['customer_id', 'customer_city', 'customer_state']], on='customer_id', how='left')
df_sup = df_sup.merge(reviews[['order_id', 'review_score']], on='order_id', how='left')  # apenas um merge

# 2. Calcular a média de entrega por vendedor
seller_orders = orders_clean.merge(order_items[['order_id', 'seller_id']], on='order_id')
seller_performance = seller_orders.groupby('seller_id').apply(
    lambda g: (g['order_delivered_customer_date'] - g['order_purchase_timestamp']).dt.days.mean()
).to_frame('seller_avg_delivery_time')
df_sup = df_sup.merge(seller_performance, on='seller_id', how='left')

# 3. Calcular a média de entrega por região (estado)
orders_with_state = orders_clean.merge(customers[['customer_id', 'customer_state']], on='customer_id')
state_performance = orders_with_state.groupby('customer_state').apply(
    lambda g: (g['order_delivered_customer_date'] - g['order_purchase_timestamp']).dt.days.mean()
).to_frame('state_avg_delivery_time')
df_sup = df_sup.merge(state_performance, on='customer_state', how='left')

# 4. Feature engineering para informações do produto
df_sup['product_volume_cm3'] = df_sup['product_length_cm'] * df_sup['product_height_cm'] * df_sup['product_width_cm']
df_sup['product_density'] = df_sup['product_weight_g'] / df_sup['product_volume_cm3']

# 5. Feature engineering para distância (usando cidade/estado como proxy)
df_sup['same_state'] = (df_sup['seller_state'] == df_sup['customer_state']).astype(int)
df_sup['same_city'] = (df_sup['seller_city'] == df_sup['customer_city']).astype(int)

# Feature engineering: prazo de entrega real x estimado (em dias)
df_sup['delivery_time'] = (df_sup['order_delivered_customer_date'] - df_sup['order_purchase_timestamp']).dt.days
df_sup['delivery_delay'] = (df_sup['order_delivered_customer_date'] - df_sup['order_estimated_delivery_date']).dt.days

# Agora sim, tratar review_score (garantindo que existe)
if 'review_score' in df_sup.columns:
    df_sup['review_score'] = df_sup['review_score'].fillna(df_sup['review_score'].mean())

# Remoção de valores nulos em todos os campos importantes
df_sup = df_sup.dropna(subset=['review_score', 'delivery_time', 'delivery_delay'])

# Preencher valores nulos nas categorias com valores de substituição adequados
if 'product_category_name_english' in df_sup.columns:
    df_sup['product_category_name_english'] = df_sup['product_category_name_english'].fillna('unknown')
if 'customer_state' in df_sup.columns:
    df_sup['customer_state'] = df_sup['customer_state'].fillna(df_sup['customer_state'].mode()[0])
if 'payment_type' in df_sup.columns:
    df_sup['payment_type'] = df_sup['payment_type'].fillna(df_sup['payment_type'].mode()[0])
if 'order_status' in df_sup.columns:
    df_sup['order_status'] = df_sup['order_status'].fillna('delivered')  # Assumindo que todos os pedidos com revisão foram entregues

# Preencher valores nulos nas variáveis numéricas com a mediana
if 'payment_value' in df_sup.columns:
    df_sup['payment_value'] = df_sup['payment_value'].fillna(df_sup['payment_value'].median())

# Selecionar features finais para modelagem
features_sup = [
    'payment_value', 'payment_type', 'customer_state', 'product_category_name_english',
    'delivery_time', 'delivery_delay', 'order_status', 'review_score',
    'product_weight_g',      # Adicionada para regressão
    'product_length_cm',     # Adicionada para regressão
    'product_height_cm',     # Adicionada para regressão
    'product_width_cm',      # Adicionada para regressão
    'freight_value',         # Adicionada para regressão
    'product_volume_cm3',    # Adicionada para regressão
    'product_density',       # Adicionada para regressão
    'seller_avg_delivery_time', # Adicionada para regressão
    'state_avg_delivery_time',  # Adicionada para regressão
    'same_state',               # Adicionada para regressão
    'same_city'                 # Adicionada para regressão
]
# A linha abaixo foi comentada pois a filtragem específica de colunas
# deve acontecer dentro dos pipelines de cada modelo, se necessário,
# ou ao preparar X_class e X_reg separadamente.
# df_sup = df_sup[features_sup]

# Verificar se ainda existem valores nulos
print("Valores nulos por coluna após tratamento:")
print(df_sup.isnull().sum())

df_sup.head()

## 5.2 Preparação para o Problema Não Supervisionado

Vamos criar um dataset agregando informações por cliente:
- Frequência de compras
- Ticket médio
- Diversidade de categorias
- Região
- Forma de pagamento mais comum

In [ ]:
# Preparação para detecção de vendedores problemáticos

# Verificar se as variáveis necessárias existem e recriá-las se necessário
try:
    # Testar se orders_clean existe
    type(orders_clean)
except NameError:
    # Se não existe, criá-la a partir de orders
    print("Criando variável orders_clean...")
    orders_clean = orders.dropna(subset=['order_delivered_customer_date'])

# 1. Métricas básicas por vendedor
df_unsup = order_items.merge(orders_clean, on='order_id')
df_unsup = df_unsup.merge(reviews[['order_id', 'review_score', 'review_comment_message']], on='order_id', how='left')

# Agregar métricas por vendedor (nivelando a hierarquia do resultado com flatten_names=True)
vendedor_metricas = df_unsup.groupby('seller_id').agg({
    'order_id': 'count',  # Volume de vendas
    'price': ['mean', 'std'],  # Média e desvio padrão dos preços
    'review_score': ['mean', 'count'],  # Média e quantidade de avaliações
    'order_status': lambda x: (x == 'canceled').mean()  # Taxa de cancelamento
}).reset_index()

# Flatten das colunas multi-índice
vendedor_metricas.columns = ['seller_id', 'total_vendas', 'preco_medio', 'preco_std', 
                            'avaliacao_media', 'num_avaliacoes', 'taxa_cancelamento']

# 2. Calcular crescimento suspeito (variação percentual mês a mês)
vendas_mensais = df_unsup.groupby(['seller_id', pd.Grouper(key='order_purchase_timestamp', freq='M')])['order_id'].count().reset_index()
vendas_mensais['crescimento'] = vendas_mensais.groupby('seller_id')['order_id'].pct_change()

# Agregar crescimento por vendedor
crescimento_suspeito = vendas_mensais.groupby('seller_id')['crescimento'].agg(['mean', 'max']).reset_index()
crescimento_suspeito.columns = ['seller_id', 'crescimento_medio', 'crescimento_maximo']

# 3. Análise geográfica das avaliações
geo_concentracao = df_unsup.merge(customers[['customer_id', 'customer_state']], on='customer_id').\
    groupby(['seller_id', 'customer_state'])['review_score'].agg(['count', 'mean']).reset_index()

# Calcular índice de concentração geográfica (% de vendas no estado principal)
geo_concentracao_idx = geo_concentracao.groupby('seller_id').apply(
    lambda x: x['count'].max() / x['count'].sum()
).reset_index()
geo_concentracao_idx.columns = ['seller_id', 'concentracao_geografica']

# 4. Juntar todas as métricas
df_unsup = vendedor_metricas.merge(crescimento_suspeito, on='seller_id', how='left')
df_unsup = df_unsup.merge(geo_concentracao_idx, on='seller_id', how='left')

# 5. Criar features para detecção de anomalias
df_unsup['preco_zscore'] = (df_unsup['preco_medio'] - df_unsup['preco_medio'].mean()) / df_unsup['preco_medio'].std()
df_unsup['avaliacao_zscore'] = (df_unsup['avaliacao_media'] - df_unsup['avaliacao_media'].mean()) / df_unsup['avaliacao_media'].std()
df_unsup['crescimento_zscore'] = (df_unsup['crescimento_maximo'] - df_unsup['crescimento_maximo'].mean()) / df_unsup['crescimento_maximo'].std()

# Criando variáveis para análise de clientes
# Frequência de compras por cliente
compras_cliente = orders_clean.groupby('customer_id')['order_id'].count().rename('freq_compras')

# Ticket médio por cliente
df_pagamento_tipo_temp = orders_clean.merge(payments, on='order_id')
pagamentos_cliente = df_pagamento_tipo_temp.groupby('customer_id')['payment_value'].mean().rename('ticket_medio')

# Diversidade de categorias
itens_cliente = orders_clean.merge(order_items, on='order_id')
itens_cliente = itens_cliente.merge(products[['product_id', 'product_category_name']], on='product_id', how='left')
diversidade = itens_cliente.groupby('customer_id')['product_category_name'].nunique().rename('num_categorias')

# Forma de pagamento mais comum
pagamento_tipo = df_pagamento_tipo_temp.groupby('customer_id')['payment_type'].agg(lambda x: x.mode()[0]).rename('payment_type')

# Região
regiao = customers.set_index('customer_id')['customer_state']

# Unir tudo
df_unsup_clientes = pd.DataFrame(compras_cliente).join([pagamentos_cliente, diversidade, pagamento_tipo, regiao])
df_unsup_clientes = df_unsup_clientes.dropna()

df_unsup_clientes.head()

Os dados estão prontos para a modelagem! A seguir, serão aplicados os algoritmos de machine learning para cada abordagem.

# 6. Modelagem Supervisionada: Previsão da Nota de Avaliação

Nesta etapa, serão aplicados três algoritmos de regressão para prever a nota de avaliação do cliente:
- Regressão Linear (baseline)
- Random Forest Regressor
- SVR (Support Vector Regressor)

Serão utilizadas validação cruzada, regularização e as métricas R², RMSE e MAE. Também serão apresentadas curvas de aprendizado e análise de overfitting/underfitting.

In [ ]:
# Pré-processamento: encoding e padronização
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

X = df_sup.drop('review_score', axis=1)
y = df_sup['delivery_time']  # Corrigido: usar a coluna já existente

# Separar variáveis categóricas e numéricas
cat_cols = ['seller_state', 'customer_state', 'product_category_name', 'same_state', 'same_city']
num_cols = ['product_weight_g', 'product_volume_cm3', 'product_density', 
            'seller_avg_delivery_time', 'state_avg_delivery_time']

# Pipeline de pré-processamento com imputação para garantir que não haja valores nulos
# Os imputadores são adicionados para maior segurança, mesmo que já tenhamos tratado os nulos
preprocessor = ColumnTransformer([
    ('num', Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ]), num_cols),
    ('cat', Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', OneHotEncoder(handle_unknown='ignore'))
    ]), cat_cols)
])

# Verificar valores nulos antes da divisão
print("Valores nulos em X antes da divisão:")
print(X.isnull().sum())

# Primeiro fazer um sample de 30% dos dados
X_sample = X.sample(frac=0.3, random_state=42)
y_sample = y[X_sample.index]

# Divisão treino-teste no sample
X_train, X_test, y_train, y_test = train_test_split(X_sample, y_sample, test_size=0.2, random_state=42)

# Verificar valores nulos após a divisão
print("Valores nulos em X_train após a divisão:")
print(X_train.isnull().sum())

print("\nTamanho do dataset original:", len(X))
print("Tamanho do sample (30%):", len(X_sample))
print("Tamanho do conjunto de treino:", len(X_train))
print("Tamanho do conjunto de teste:", len(X_test))

In [ ]:
# Função para avaliação dos modelos de regressão
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def avaliar_regressao(model, X_train, X_test, y_train, y_test):
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    print('Treino:')
    print('R²:', r2_score(y_train, y_pred_train))
    print('RMSE:', np.sqrt(mean_squared_error(y_train, y_pred_train)))
    print('MAE:', mean_absolute_error(y_train, y_pred_train))
    print('\nTeste:')
    print('R²:', r2_score(y_test, y_pred_test))
    print('RMSE:', np.sqrt(mean_squared_error(y_test, y_pred_test)))
    print('MAE:', mean_absolute_error(y_test, y_pred_test))
    return y_pred_train, y_pred_test

In [ ]:
# 1. Regressão Linear (baseline)
from sklearn.linear_model import LinearRegression

pipe_lr = Pipeline([
    ('pre', preprocessor),
    ('reg', LinearRegression())
])
pipe_lr.fit(X_train, y_train)
print('Regressão Linear:')
y_pred_train_lr, y_pred_test_lr = avaliar_regressao(pipe_lr, X_train, X_test, y_train, y_test)

# Comentário explicativo sobre a resolução do problema de valores NaN
print("\nNota sobre o tratamento de valores ausentes:")
print("Os valores NaN foram tratados em três níveis para garantir robustez:")
print("1. No dataframe: preenchimento específico para cada coluna")
print("2. No pré-processamento: imputação por mediana (numéricas) e valor mais frequente (categóricas)")
print("3. No OneHotEncoder: parâmetro handle_unknown='ignore' para lidar com categorias ausentes")

In [ ]:
# 2. Random Forest Regressor
from sklearn.ensemble import RandomForestRegressor

pipe_rf = Pipeline([
    ('pre', preprocessor),
    ('reg', RandomForestRegressor(n_estimators=100, random_state=42))
])
pipe_rf.fit(X_train, y_train)
print('Random Forest:')
y_pred_train_rf, y_pred_test_rf = avaliar_regressao(pipe_rf, X_train, X_test, y_train, y_test)

In [ ]:
# 3. SVR (Support Vector Regressor)
from sklearn.svm import SVR

pipe_svr = Pipeline([
    ('pre', preprocessor),
    ('reg', SVR())
])
pipe_svr.fit(X_train, y_train)
print('SVR:')
y_pred_train_svr, y_pred_test_svr = avaliar_regressao(pipe_svr, X_train, X_test, y_train, y_test)

In [ ]:
# Curvas de aprendizado para Random Forest
from sklearn.model_selection import learning_curve

# Verificar as dimensões dos dados após o pré-processamento
# Isso é útil para confirmar que não perdemos observações devido a valores NaN
X_processed = preprocessor.transform(X)
print(f"\nVerificação final de dimensões:")
print(f"X original: {X.shape}")
print(f"X após pré-processamento: {X_processed.shape}")
print(f"Isso confirma que o tratamento de valores ausentes está funcionando corretamente.")

train_sizes, train_scores, test_scores = learning_curve(
    pipe_rf, X, y, cv=5, scoring='neg_root_mean_squared_error', n_jobs=-1,
    train_sizes=np.linspace(0.1, 1.0, 5), random_state=42
)

train_scores_mean = -train_scores.mean(axis=1)
test_scores_mean = -test_scores.mean(axis=1)

plt.figure(figsize=(8,5))
plt.plot(train_sizes, train_scores_mean, label='Treino')
plt.plot(train_sizes, test_scores_mean, label='Validação')
plt.xlabel('Tamanho do Treinamento')
plt.ylabel('RMSE')
plt.title('Curva de Aprendizado - Random Forest')
plt.legend()
plt.show()

Os resultados dos modelos supervisionados serão comparados e discutidos na próxima etapa. Em seguida, será realizada a modelagem não supervisionada (clustering).

# 7. Modelagem Não Supervisionada: Clustering de Clientes

Nesta etapa, serão aplicadas três técnicas de clustering para segmentação de clientes:
- KMeans
- DBSCAN
- Agglomerative Clustering

Também será utilizada a técnica de redução de dimensionalidade (PCA) para visualização dos clusters. As métricas de avaliação incluem Silhouette Score, Davies-Bouldin e Inércia (quando aplicável).

In [ ]:
# Pré-processamento: encoding e padronização para clustering
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

# Fazer sample de 30% dos dados para clustering
X_unsup = df_unsup.sample(frac=0.3, random_state=42).copy()
print(f"Tamanho do dataset original: {len(df_unsup)}")
print(f"Tamanho do sample (30%): {len(X_unsup)}")

# Features para detecção de anomalias em vendedores
anomaly_features = [
    'total_vendas', 'preco_medio', 'preco_std', 'avaliacao_media', 
    'taxa_cancelamento', 'crescimento_medio', 'crescimento_maximo',
    'concentracao_geografica', 'preco_zscore', 'avaliacao_zscore', 'crescimento_zscore'
]

# Preparar features para detecção de anomalias
anomaly_features = [
    'total_vendas', 'preco_medio', 'preco_std', 'avaliacao_media', 
    'taxa_cancelamento', 'crescimento_medio', 'crescimento_maximo',
    'concentracao_geografica', 'preco_zscore', 'avaliacao_zscore', 'crescimento_zscore'
]

# Preprocessamento para detecção de anomalias
preprocessor_unsup = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Verificar valores nulos antes do processamento
print("Valores nulos em X_unsup antes do processamento:")
print(X_unsup[anomaly_features].isnull().sum())

# Preparar os dados para clustering
X_unsup_proc = preprocessor_unsup.fit_transform(X_unsup[anomaly_features])

In [ ]:
# Redução de dimensionalidade para visualização
from sklearn.decomposition import PCA

pca = PCA(n_components=2, random_state=42)
X_pca = pca.fit_transform(X_unsup_proc)
plt.figure(figsize=(7,5))
plt.scatter(X_pca[:,0], X_pca[:,1], alpha=0.3)
plt.title('Clientes no Espaço PCA (sem cluster)')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.show()

In [ ]:
# 1. KMeans
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=4, random_state=42)
labels_kmeans = kmeans.fit_predict(X_unsup_proc)

plt.figure(figsize=(7,5))
plt.scatter(X_pca[:,0], X_pca[:,1], c=labels_kmeans, cmap='tab10', alpha=0.5)
plt.title('Clusters de Clientes - KMeans')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.show()

# Converter matriz esparsa para array denso se necessário
if hasattr(X_unsup_proc, "toarray"):
    X_unsup_proc_dense = X_unsup_proc.toarray()
else:
    X_unsup_proc_dense = X_unsup_proc

print('Silhouette Score:', silhouette_score(X_unsup_proc, labels_kmeans))
print('Davies-Bouldin:', davies_bouldin_score(X_unsup_proc_dense, labels_kmeans))
print('Inércia:', kmeans.inertia_)

In [ ]:
# 2. DBSCAN
from sklearn.cluster import DBSCAN
import numpy as np
import matplotlib.pyplot as plt

dbscan = DBSCAN(eps=1.5, min_samples=10)
labels_dbscan = dbscan.fit_predict(X_unsup_proc)

# Create figure and axes for the plot
fig, ax = plt.subplots(figsize=(7,5))

# Generate the scatter plot
scatter_plot = ax.scatter(X_pca[:,0], X_pca[:,1], c=labels_dbscan, cmap='tab10', alpha=0.5)

# Set title and axis labels
ax.set_title('Clusters de Clientes - DBSCAN')
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')

# Create legend
# Get unique sorted cluster labels
unique_labels_sorted = sorted(np.unique(labels_dbscan))

# Get the actual colors used by scatter plot for each unique label
# PathCollection (scatter_plot) is a ScalarMappable, so to_rgba works.
colors_for_labels = scatter_plot.to_rgba(unique_labels_sorted)

legend_handles = []
for i, label_val in enumerate(unique_labels_sorted):
    legend_text = f'Cluster {label_val}' if label_val != -1 else 'Ruído' # "Ruído" for Noise
    legend_handles.append(plt.Line2D([0], [0], marker='o', color='w', # Invisible line
                                     markerfacecolor=colors_for_labels[i],
                                     markersize=8,
                                     label=legend_text))

ax.legend(handles=legend_handles, title="Legenda")

plt.show()

# Filtrar ruído para métricas
mask = labels_dbscan != -1
if mask.sum() > 1: # Ensure there are at least 2 points in non-noise clusters
    # Convert matriz esparsa para array denso se necessário para Davies-Bouldin
    if hasattr(X_unsup_proc, "toarray"):
        X_unsup_proc_dense = X_unsup_proc.toarray()
    else:
        X_unsup_proc_dense = X_unsup_proc
    
    # Check if there are enough unique cluster labels (excluding noise) for metrics
    unique_cluster_labels = np.unique(labels_dbscan[mask])
    if len(unique_cluster_labels) > 1:
        print('Silhouette Score:', silhouette_score(X_unsup_proc[mask], labels_dbscan[mask]))
        print('Davies-Bouldin:', davies_bouldin_score(X_unsup_proc_dense[mask], labels_dbscan[mask]))
    else:
        print('Não há clusters suficientes (excluindo ruído) para calcular Silhouette Score ou Davies-Bouldin Score.')
else:
    print('Poucos clusters ou apenas ruído encontrado pelo DBSCAN. Métricas de cluster não calculadas.')

In [ ]:
# 3. Agglomerative Clustering
from sklearn.cluster import AgglomerativeClustering

agg = AgglomerativeClustering(n_clusters=4)
labels_agg = agg.fit_predict(X_unsup_proc)

plt.figure(figsize=(7,5))
plt.scatter(X_pca[:,0], X_pca[:,1], c=labels_agg, cmap='tab10', alpha=0.5)
plt.title('Clusters de Clientes - Agglomerative')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.show()

# Converter matriz esparsa para array denso se necessário
if hasattr(X_unsup_proc, "toarray"):
    X_unsup_proc_dense = X_unsup_proc.toarray()
else:
    X_unsup_proc_dense = X_unsup_proc

print('Silhouette Score:', silhouette_score(X_unsup_proc, labels_agg))
print('Davies-Bouldin:', davies_bouldin_score(X_unsup_proc_dense, labels_agg))

Os resultados dos clusters serão analisados e comparados na próxima etapa, junto com a discussão dos insights de negócio extraídos.

# 8. Avaliação Comparativa e Insights de Negócio

Nesta etapa, comparamos o desempenho dos modelos supervisionados e dos clusters, discutindo os principais resultados, limitações e implicações para o negócio Olist.

## 8.1 Avaliação dos Modelos Supervisionados

- **Regressão Linear:** Serve como baseline. Resultados esperados: desempenho modesto, sensível a outliers e relações não-lineares.
- **Random Forest:** Geralmente apresenta melhor desempenho, lida bem com não-linearidades e variáveis categóricas. Pode apresentar overfitting se não regularizado.
- **SVR:** Útil para capturar relações complexas, mas pode ser sensível à escala dos dados e ao tuning de hiperparâmetros.

**Comparação das métricas:**
- R², RMSE e MAE em treino e teste.
- Curva de aprendizado: análise de overfitting/underfitting.

**Principais insights:**
- Quais variáveis mais impactam a nota do cliente?
- O modelo consegue antecipar avaliações baixas com boa precisão?
- Possíveis melhorias: tuning, mais features, outros algoritmos.

## 8.2 Avaliação dos Modelos Não Supervisionados

- **KMeans:** Permite identificar grupos bem definidos, útil para segmentação de marketing.
- **DBSCAN:** Detecta grupos de clientes "fora da curva" (anomalias), mas pode formar poucos clusters dependendo dos parâmetros.
- **Agglomerative:** Útil para hierarquias e dendrogramas, pode revelar subgrupos interessantes.

**Comparação das métricas:**
- Silhouette Score, Davies-Bouldin, Inércia (KMeans).
- Visualização dos clusters no espaço PCA.

**Principais insights:**
- Existem grupos de clientes recorrentes, de alto ticket ou caçadores de promoções?
- Como as regiões e formas de pagamento se distribuem entre os clusters?
- Possíveis ações: campanhas segmentadas, ofertas personalizadas, retenção de clientes valiosos.

# 9. Conclusões Finais

- O projeto demonstrou a aplicação de técnicas de machine learning supervisionado e não supervisionado no contexto do e-commerce brasileiro.
- Modelos supervisionados permitem antecipar avaliações negativas e atuar preventivamente na experiência do cliente.
- Modelos de clustering revelam perfis distintos de clientes, apoiando estratégias de marketing e retenção.
- O uso de múltiplos algoritmos e métricas garante robustez e confiabilidade nas análises.
- Recomenda-se aprofundar o tuning dos modelos, explorar mais features e aplicar as soluções em ambiente real para maximizar o valor de negócio.

**Obrigado!**

In [ ]:
# 6. Model Evaluation and Performance Analysis
from sklearn.metrics import explained_variance_score, make_scorer
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import seaborn as sns

# Configurações visuais para melhorar a aparência dos gráficos
plt.style.use('ggplot')
sns.set_palette('viridis')
sns.set_context("notebook", font_scale=1.2)

# Garantir que pipe_rf está definido e treinado
try:
    pipe_rf
except NameError:
    from sklearn.ensemble import RandomForestRegressor
    pipe_rf = Pipeline([
        ('pre', preprocessor),
        ('reg', RandomForestRegressor(n_estimators=100, random_state=42))
    ])
    pipe_rf.fit(X_train, y_train)

# Obter nomes das features após o pré-processamento
num_features = preprocessor.named_transformers_['num'].get_feature_names_out(num_cols)
cat_features = preprocessor.named_transformers_['cat'].named_steps['encoder'].get_feature_names_out(cat_cols)
feature_names = list(num_features) + list(cat_features)

# Garantir que o tamanho bate com o feature_importances_
print('Nomes de features:', len(feature_names), '| Importâncias:', len(pipe_rf.named_steps['reg'].feature_importances_))

feature_importance = pd.DataFrame({
    'feature': feature_names,
    'importance': pipe_rf.named_steps['reg'].feature_importances_
})
feature_importance = feature_importance.sort_values('importance', ascending=False)

# 1. Gráfico melhorado de importância das features - Top 15 features apenas para maior clareza
plt.figure(figsize=(12, 8))
top_features = feature_importance.head(15)
ax = sns.barplot(data=top_features, x='importance', y='feature', palette='viridis')

# Adicionar valores nas barras para melhor legibilidade
for i, v in enumerate(top_features['importance']):
    ax.text(v + 0.01, i, f'{v:.3f}', va='center')

plt.title('Top 15 Features - Importância no Modelo Random Forest', fontsize=16)
plt.xlabel('Importância', fontsize=14)
plt.ylabel('Feature', fontsize=14)
plt.tight_layout()
plt.show()

# Calculate explained variance (R²) using cross-validation
cv_scores = cross_val_score(pipe_rf, X_train, y_train, cv=5, scoring='r2')
print("\nCross-validation R² scores:", cv_scores)
print("Mean R²:", cv_scores.mean())
print("Standard deviation of R²:", cv_scores.std())

# Calculate residuals
y_pred = pipe_rf.predict(X_test)
residuals = y_test - y_pred

# 2. Gráfico melhorado de resíduos vs valores previstos
plt.figure(figsize=(12, 8))

# Criar um scatter plot com um colormap representando a densidade de pontos
# Isso ajuda a visualizar melhor os padrões quando há muitos pontos
sc = plt.scatter(y_pred, residuals, alpha=0.5, c=residuals, cmap='coolwarm', s=50, edgecolor='k', linewidth=0.5)
plt.colorbar(sc, label='Valor do Resíduo')

# Adicionar linha horizontal em y=0
plt.axhline(y=0, color='r', linestyle='--', linewidth=2, label='Resíduo Zero')

# Adicionar limites de +/- 2 desvios padrão
std_resid = np.std(residuals)
plt.axhline(y=2*std_resid, color='gray', linestyle=':', linewidth=1, label='+2 Desvios Padrão')
plt.axhline(y=-2*std_resid, color='gray', linestyle=':', linewidth=1, label='-2 Desvios Padrão')

# Adicionar texto explicativo para os outliers
outliers = sum((residuals > 2*std_resid) | (residuals < -2*std_resid))
pct_outliers = (outliers / len(residuals)) * 100
plt.annotate(f'Outliers: {outliers} ({pct_outliers:.1f}%)', 
             xy=(0.02, 0.95), xycoords='axes fraction',
             bbox=dict(boxstyle='round,pad=0.5', facecolor='white', alpha=0.8))

plt.title('Resíduos vs Valores Previstos', fontsize=16)
plt.xlabel('Tempo de Entrega Previsto (dias)', fontsize=14)
plt.ylabel('Resíduos (dias)', fontsize=14)
plt.legend(loc='lower right')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# 3. Histograma melhorado da distribuição dos resíduos
plt.figure(figsize=(12, 8))

# Usar distplot com mais bins para melhor visualização da distribuição
# O uso de KDE ajuda a visualizar a forma da distribuição
sns.histplot(residuals, bins=50, kde=True, color='skyblue', stat='density')

# Adicionar uma curva normal teórica para comparação
import scipy.stats as stats
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = stats.norm.pdf(x, np.mean(residuals), np.std(residuals))
plt.plot(x, p, 'k--', linewidth=2, label='Distribuição Normal Teórica')

# Adicionar linhas verticais para a média e medianas
plt.axvline(np.mean(residuals), color='red', linestyle='-', linewidth=2, label=f'Média: {np.mean(residuals):.2f}')
plt.axvline(np.median(residuals), color='green', linestyle='-', linewidth=2, label=f'Mediana: {np.median(residuals):.2f}')

# Destacar os quantis para análise da distribuição
for q, color, label in zip([0.25, 0.75], ['purple', 'orange'], ['Q1', 'Q3']):
    quantile_val = np.quantile(residuals, q)
    plt.axvline(quantile_val, color=color, linestyle=':', linewidth=1.5, 
                label=f'{label}: {quantile_val:.2f}')

plt.title('Distribuição dos Resíduos', fontsize=16)
plt.xlabel('Valor dos Resíduos', fontsize=14)
plt.ylabel('Densidade', fontsize=14)
plt.legend(loc='upper right')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# 4. Adicionar um novo gráfico: Q-Q plot para verificar normalidade dos resíduos
plt.figure(figsize=(10, 10))
from scipy import stats
stats.probplot(residuals, dist="norm", plot=plt)
plt.title('Q-Q Plot dos Resíduos', fontsize=16)
plt.xlabel('Quantis Teóricos', fontsize=14)
plt.ylabel('Quantis Amostrais', fontsize=14)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Print summary statistics of residuals com formatação melhorada
print("\nEstatísticas dos Resíduos:")
res_stats = pd.Series(residuals).describe()
for stat, value in zip(res_stats.index, res_stats.values):
    print(f"{stat.capitalize():>15}: {value:.4f}")

# 5. Box plot dos resíduos agrupados por mesma região (same_state)
# Este gráfico adicional mostra como os resíduos se comportam por grupo
test_df = pd.DataFrame({'residuals': residuals, 'same_state': X_test['same_state']})
plt.figure(figsize=(10, 6))
sns.boxplot(data=test_df, x='same_state', y='residuals', palette='Set2')
plt.axhline(y=0, color='red', linestyle='--', linewidth=1.5)
plt.title('Distribuição dos Resíduos por Estado', fontsize=16)
plt.xlabel('Mesmo Estado (1) vs Estados Diferentes (0)', fontsize=14)
plt.ylabel('Resíduos', fontsize=14)
plt.tight_layout()
plt.show()

# Análise de Desempenho do Modelo

## Principais Descobertas:

1. **Importância das Features**
   - As características mais importantes para prever o tempo de entrega são:
   - Fatores geográficos (mesmo estado, mesma cidade)
   - Características do produto (densidade, volume)
   - Métricas históricas de desempenho (tempo médio de entrega do vendedor)

2. **Desempenho do Modelo**
   - O modelo Random Forest demonstrou bom poder preditivo com:
   - Pontuações R² consistentes em todas as dobras de validação cruzada
   - Distribuição de resíduos relativamente simétrica
   - Ausência de padrões fortes nos resíduos versus valores previstos

3. **Insights de Negócio**
   - O tempo de entrega é fortemente influenciado por:
     - Proximidade geográfica entre vendedor e cliente
     - Características físicas do produto
     - Histórico de desempenho do vendedor
   - Esses insights podem ser utilizados para:
     - Otimizar o pareamento vendedor-cliente
     - Estabelecer estimativas de entrega mais precisas
     - Identificar áreas para melhoria logística

4. **Recomendações**
   - Focar na expansão da rede de vendedores em estados mal atendidos
   - Considerar características do produto ao definir estimativas de entrega
   - Utilizar o histórico de desempenho do vendedor para melhorar expectativas do cliente
   - Implementar melhorias direcionadas para regiões com baixo desempenho

In [ ]:
# Detailed Cluster Analysis

# Calculate cluster characteristics
cluster_stats = pd.DataFrame()
cluster_stats['size'] = pd.Series(labels_kmeans).value_counts()
cluster_stats['percentage'] = cluster_stats['size'] / len(labels_kmeans) * 100

# Calculate mean values for each feature by cluster
cluster_features = pd.DataFrame(X_unsup_proc, columns=anomaly_features)
cluster_features['cluster'] = labels_kmeans
cluster_means = cluster_features.groupby('cluster').mean()

# Criar um gráfico 3D interativo que mostra relação entre tempo de entrega, valor do frete e peso do produto
# Primeiro, vamos preparar um dataset com essas informações
delivery_data = df_sup[['delivery_time', 'freight_value', 'product_weight_g', 'same_state']].copy()
delivery_data['same_state_label'] = delivery_data['same_state'].map({1: 'Mesmo Estado', 0: 'Estados Diferentes'})

# Criar o gráfico 3D interativo com plotly
fig = px.scatter_3d(
    delivery_data.sample(1000), # Amostra para melhorar a performance
    x='delivery_time',
    y='freight_value',
    z='product_weight_g',
    color='same_state_label',
    opacity=0.7,
    title='Relação entre Tempo de Entrega, Valor do Frete e Peso do Produto',
    labels={
        'delivery_time': 'Tempo de Entrega (dias)',
        'freight_value': 'Valor do Frete (R$)',
        'product_weight_g': 'Peso do Produto (g)',
        'same_state_label': 'Localização'
    },
    color_discrete_sequence=px.colors.qualitative.Set1
)

# Personalizar o layout
fig.update_layout(
    scene=dict(
        xaxis_title='Tempo de Entrega (dias)',
        yaxis_title='Valor do Frete (R$)',
        zaxis_title='Peso do Produto (g)',
    ),
    legend_title_text='Localização',
    margin=dict(l=0, r=0, b=0, t=40),
)

# Adicionar uma linha de tendência para melhor visualização
x_lines = delivery_data['delivery_time'].sample(20)
y_lines = delivery_data['freight_value'].sample(20)
z_lines = delivery_data['product_weight_g'].sample(20)

# Ajustar um plano 3D aos dados para visualizar tendências
from sklearn.linear_model import LinearRegression
model = LinearRegression()
X_plane = delivery_data[['delivery_time', 'freight_value']].sample(500)
y_plane = delivery_data['product_weight_g'].iloc[X_plane.index]
model.fit(X_plane, y_plane)

# Criar uma grade para o plano
xx, yy = np.meshgrid(
    np.linspace(delivery_data['delivery_time'].min(), delivery_data['delivery_time'].max(), 10),
    np.linspace(delivery_data['freight_value'].min(), delivery_data['freight_value'].max(), 10)
)
zz = model.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)

# Adicionar o plano ao gráfico
fig.add_trace(go.Surface(
    x=xx, y=yy, z=zz,
    colorscale='Viridis',
    opacity=0.5,
    showscale=False
))

# Exibir o gráfico interativo
fig.show()

# Visualization of cluster characteristics
plt.figure(figsize=(15, 6))

# Plot 2: Feature Means by Cluster
plt.subplot(1, 1, 1)
sns.heatmap(cluster_means, cmap='YlOrRd', annot=True, fmt='.2f')
plt.title('Mean Feature Values by Cluster')
plt.tight_layout()
plt.show()

# Print cluster insights
print("\nCluster Characteristics:")
for cluster in range(len(cluster_stats)):
    print(f"\nCluster {cluster}:")
    print(f"Size: {cluster_stats.loc[cluster, 'size']} ({cluster_stats.loc[cluster, 'percentage']:.1f}%)")
    print("Key characteristics:")
    # Get top 3 distinctive features for this cluster
    distinctive_features = cluster_means.loc[cluster].sort_values(ascending=False)[:3]
    for feat, val in distinctive_features.items():
        print(f"- High {feat}: {val:.2f}")

# Evaluate cluster quality
if hasattr(kmeans, 'inertia_'):
    print(f"\nCluster Inertia: {kmeans.inertia_:.2f}")
print(f"Silhouette Score: {silhouette_score(X_unsup_proc, labels_kmeans):.2f}")
print(f"Davies-Bouldin Score: {davies_bouldin_score(X_unsup_proc_dense, labels_kmeans):.2f}")

# Conclusões Finais e Recomendações

## Aprendizado Supervisionado: Previsão do Tempo de Entrega

1. **Desempenho do Modelo**
   - O modelo Random Forest alcançou bom desempenho preditivo
   - Características geográficas e do produto são os preditores mais fortes
   - O modelo pode estimar tempos de entrega de forma confiável dentro de margens razoáveis

2. **Impacto para o Negócio**
   - Estimativas de entrega mais precisas podem melhorar a satisfação do cliente
   - Insights sobre os principais fatores de tempo de entrega permitem melhorias direcionadas
   - Potencial para previsões dinâmicas de tempo de entrega baseadas em dados em tempo real

## Aprendizado Não Supervisionado: Segmentação de Vendedores

1. **Análise de Clusters**
   - Identificados segmentos distintos de vendedores com características únicas
   - Encontrados padrões potencialmente problemáticos em alguns clusters de vendedores
   - Concentração geográfica e métricas de desempenho revelam oportunidades de otimização

2. **Recomendações para o Negócio**
   - Implementar intervenções direcionadas para clusters de vendedores de alto risco
   - Desenvolver programas de melhoria de desempenho para vendedores com base nas características do cluster
   - Usar insights de clustering para integração e monitoramento de vendedores

## Próximos Passos

1. **Melhorias no Modelo**
   - Incorporar mais características (clima, sazonalidade, etc.)
   - Implementar atualizações de previsão em tempo real
   - Retreinamento regular do modelo com novos dados

2. **Implementação no Negócio**
   - Integrar previsões nos sistemas voltados para o cliente
   - Desenvolver monitoramento automatizado de vendedores com base na análise de clusters
   - Criar dashboard para acompanhamento de métricas-chave e anomalias

3. **Monitoramento e Manutenção**
   - Configurar monitoramento regular do desempenho do modelo
   - Acompanhar KPIs de negócio impactados pelos modelos
   - Coletar feedback continuamente para futuras melhorias

In [ ]:
# Visualizações aprimoradas para análise de cluster
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import silhouette_samples
import plotly.express as px
import plotly.graph_objects as go
from matplotlib.colors import ListedColormap
import numpy as np

# 1. Visualização da distribuição de tamanhos dos clusters com gráfico interativo
cluster_counts = pd.Series(labels_kmeans).value_counts().sort_index()
cluster_percentages = (cluster_counts / len(labels_kmeans) * 100).round(1)
labels = [f'Cluster {i}\n{p}%' for i, p in enumerate(cluster_percentages)]

# Visualização com Matplotlib
plt.figure(figsize=(10, 6))
colors = plt.cm.tab10(np.linspace(0, 1, len(cluster_counts)))
plt.pie(cluster_counts, labels=labels, autopct='%1.1f%%', startangle=90, colors=colors,
        wedgeprops={'width': 0.5, 'edgecolor': 'w', 'linewidth': 2})
plt.title('Cluster Size Distribution', fontsize=16)
plt.tight_layout()
plt.show()

# 2. Visualização aprimorada de features por cluster com heatmap interativo
cluster_features = pd.DataFrame(X_unsup_proc, columns=anomaly_features)
cluster_features['cluster'] = labels_kmeans
cluster_means = cluster_features.groupby('cluster').mean()

# Normalizar o heatmap para melhor visualização
cluster_means_normalized = (cluster_means - cluster_means.mean()) / cluster_means.std()
plt.figure(figsize=(12, 8))
ax = sns.heatmap(cluster_means_normalized, cmap='RdBu_r', annot=True, fmt='.2f', 
            linewidths=.5, center=0, cbar_kws={'label': 'Valor Normalizado (Z-score)'})
plt.title('Mean Feature Values by Cluster', fontsize=16)
plt.xlabel('Features', fontsize=12)
plt.ylabel('Clusters', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

# 3. Análise silhouette para avaliação da qualidade dos clusters
silhouette_vals = silhouette_samples(X_unsup_proc, labels_kmeans)
y_ticks = []
y_lower, y_upper = 0, 0

fig, ax = plt.subplots(figsize=(10, 8))
cm = plt.cm.get_cmap('tab10')

for i, cluster in enumerate(np.unique(labels_kmeans)):
    cluster_silhouette_vals = silhouette_vals[labels_kmeans == cluster]
    cluster_silhouette_vals.sort()
    y_upper += len(cluster_silhouette_vals)
    color = cm(float(i) / len(np.unique(labels_kmeans)))
    ax.barh(range(y_lower, y_upper), cluster_silhouette_vals, height=1.0, 
            edgecolor='none', color=color, alpha=0.8)
    y_ticks.append((y_lower + y_upper) / 2)
    y_lower += len(cluster_silhouette_vals)

silhouette_avg = silhouette_score(X_unsup_proc, labels_kmeans)
ax.axvline(silhouette_avg, color='red', linestyle='--', 
           label=f'Média: {silhouette_avg:.2f}')
ax.set_yticks(y_ticks)
ax.set_yticklabels([f'Cluster {i}' for i in range(len(np.unique(labels_kmeans)))])
ax.set_xlabel('Silhouette Coefficient', fontsize=12)
ax.set_ylabel('Cluster', fontsize=12)
ax.set_title('Análise Silhouette por Cluster', fontsize=16)
ax.legend(loc='lower right')
plt.tight_layout()
plt.show()

# 4. Visualização 3D dos clusters com PCA para 3 componentes
pca_3d = PCA(n_components=3, random_state=42)
X_pca_3d = pca_3d.fit_transform(X_unsup_proc)

# Criando um DataFrame para facilitar a plotagem
df_pca_3d = pd.DataFrame(X_pca_3d, columns=['PC1', 'PC2', 'PC3'])
df_pca_3d['Cluster'] = labels_kmeans

# Exibir a variância explicada por cada componente
explained_variance = pca_3d.explained_variance_ratio_ * 100
print(f"Variância explicada pelas componentes principais:")
for i, variance in enumerate(explained_variance):
    print(f"PC{i+1}: {variance:.2f}%")
print(f"Variância total explicada: {sum(explained_variance):.2f}%")

# Criando uma visualização 3D interativa usando plotly.express
fig = px.scatter_3d(
    df_pca_3d,
    x='PC1',
    y='PC2',
    z='PC3',
    color='Cluster',  # Colorir os pontos com base no cluster
    opacity=0.7,
    title='Visualização 3D dos Clusters com PCA (Interativo)',
    labels={
        'PC1': f'PC1 ({explained_variance[0]:.2f}%)',
        'PC2': f'PC2 ({explained_variance[1]:.2f}%)',
        'PC3': f'PC3 ({explained_variance[2]:.2f}%)'
    },
    color_discrete_sequence=px.colors.qualitative.Set1
)

# Atualizar layout para um melhor visual
fig.update_layout(
    margin=dict(l=0, r=0, b=0, t=40),
    legend_title_text='Clusters',
    scene=dict(
        xaxis_title=f'PC1 ({explained_variance[0]:.2f}%)',
        yaxis_title=f'PC2 ({explained_variance[1]:.2f}%)',
        zaxis_title=f'PC3 ({explained_variance[2]:.2f}%)'
    )
)

# Exibir o gráfico interativo
fig.show()


# fig = plt.figure(figsize=(10, 8))
# ax = fig.add_subplot(111, projection='3d')
#     
# colors = plt.cm.tab10(np.linspace(0, 1, len(np.unique(labels_kmeans))))
# cmap = ListedColormap(colors)
#     
# scatter = ax.scatter(df_pca_3d['PC1'], df_pca_3d['PC2'], df_pca_3d['PC3'], 
#                      c=df_pca_3d['Cluster'], cmap=cmap, s=30, alpha=0.7)
#     
# # Adicionar uma legenda
# legend1 = ax.legend(*scatter.legend_elements(), title="Clusters")
# ax.add_artist(legend1)
#     
# # Rótulos e título
# ax.set_xlabel(f'PC1 ({explained_variance[0]:.2f}%)', fontsize=12)
# ax.set_ylabel(f'PC2 ({explained_variance[1]:.2f}%)', fontsize=12)
# ax.set_zlabel(f'PC3 ({explained_variance[2]:.2f}%)', fontsize=12)
# ax.set_title('Visualização 3D dos Clusters com PCA', fontsize=16)
#     
# plt.tight_layout()
# plt.show()

# 5. Análise das características distintivas de cada cluster
# Determinar as características mais distintivas por cluster
cluster_importance = pd.DataFrame()

for cluster in sorted(np.unique(labels_kmeans)):
    # Calcular a diferença entre a média do cluster e a média global para cada feature
    cluster_diff = cluster_means.loc[cluster] - cluster_features[anomaly_features].mean()
    # Normalizar pela desviação padrão global
    cluster_diff_norm = cluster_diff / cluster_features[anomaly_features].std()
    # Adicionar ao DataFrame
    cluster_importance[f'Cluster {cluster}'] = cluster_diff_norm

# Visualizar as características mais importantes por cluster
plt.figure(figsize=(14, 10))
sns.heatmap(cluster_importance, annot=True, cmap='RdBu_r', fmt='.2f', center=0,
           linewidths=.5, cbar_kws={'label': 'Diferença Normalizada (Z-score)'})
plt.title('Características Distintivas por Cluster', fontsize=16)
plt.xlabel('Cluster', fontsize=12)
plt.ylabel('Features', fontsize=12)
plt.tight_layout()
plt.show()

# 6. Sumário interpretativo dos clusters
print("\n===== INTERPRETAÇÃO DOS CLUSTERS =====\n")

for cluster in sorted(np.unique(labels_kmeans)):
    # Selecionar as 3 características mais positivas e negativas para este cluster
    cluster_features_sorted = cluster_importance[f'Cluster {cluster}'].sort_values()
    top_negative = cluster_features_sorted.head(3)
    top_positive = cluster_features_sorted.tail(3)
    
    # Calcular o tamanho e porcentagem do cluster
    cluster_size = sum(labels_kmeans == cluster)
    cluster_percent = (cluster_size / len(labels_kmeans) * 100)
    
    print(f"Cluster {cluster} ({cluster_size} elementos, {cluster_percent:.1f}% do total):")
    print(f"  Características distintivas positivas:")
    for feat, val in top_positive.items():
        print(f"    - {feat}: {val:.2f} desvios padrão acima da média")
    print(f"  Características distintivas negativas:")
    for feat, val in top_negative.items():
        print(f"    - {feat}: {val:.2f} desvios padrão abaixo da média")
    print()

# Análise Detalhada dos Clusters de Vendedores

## Características dos Clusters Identificados

Após a aplicação do algoritmo K-Means, identificamos quatro clusters distintos de vendedores com as seguintes características:

### Cluster 0 (11,1% dos vendedores):
- **Pontos fortes**: Concentração geográfica elevada (0,38), indicando operação regionalizada
- **Pontos fracos**: Avaliações muito baixas (Z-score -2,11), sugerindo problemas de qualidade
- **Interpretação**: "Vendedores regionais com problemas de satisfação do cliente" - Este grupo opera em regiões específicas com baixo desempenho em avaliações

### Cluster 1 (77,0% dos vendedores):
- **Pontos fortes**: Avaliações positivas (Z-score 0,32), boa distribuição geográfica
- **Pontos fracos**: Crescimento moderado e preços medianos
- **Interpretação**: "Vendedores regulares" - Representam a maioria do marketplace, com desempenho estável e avaliações satisfatórias

### Cluster 2 (7,9% dos vendedores):
- **Pontos fortes**: Crescimento extraordinário (Z-score 2,49), alto volume de vendas (1,53)
- **Pontos fracos**: Concentração geográfica negativa (-0,59), sugerindo distribuição muito ampla
- **Interpretação**: "Vendedores em expansão rápida" - Grupo em crescimento acelerado, possivelmente com estratégias agressivas de expansão

### Cluster 3 (4,0% dos vendedores):
- **Pontos fortes**: Preços muito elevados (Z-score 3,36), alto desvio padrão nos preços (3,14)
- **Pontos fracos**: Crescimento limitado (-0,27)
- **Interpretação**: "Vendedores premium" - Focados em produtos de alto valor, possivelmente em nichos específicos

## Métricas de Qualidade dos Clusters

- **Silhouette Score**: 0,38 - Indica separação moderadamente boa entre clusters
- **Davies-Bouldin Score**: 1,14 - Valor relativamente baixo, sugerindo clusters bem definidos
- **Inércia**: 5394,03 - Medida de compactação interna dos clusters

## Visualização dos Clusters

A visualização 3D dos clusters através de PCA mostra uma boa separação espacial, com:
- **PC1**: 26,94% da variância explicada - Relacionada principalmente a preço e volume
- **PC2**: 21,70% da variância explicada - Relacionada a avaliações e crescimento
- **PC3**: 20,02% da variância explicada - Relacionada a concentração geográfica
- **Variância total explicada**: 68,66% - Um bom percentual para visualização e interpretação

## Recomendações Estratégicas por Cluster

### Para Cluster 0 (Vendedores problemáticos):
- Implementar programa de melhoria de qualidade com monitoramento rigoroso
- Oferecer treinamentos específicos sobre atendimento ao cliente
- Avaliar possível descontinuação se não houver melhora

### Para Cluster 1 (Vendedores regulares):
- Incentivar crescimento com programas de fidelidade
- Fornecer ferramentas de automação para escalar operações
- Reconhecer desempenho consistente com benefícios no marketplace

### Para Cluster 2 (Vendedores em expansão):
- Monitorar qualidade durante o crescimento acelerado
- Oferecer suporte logístico para manter a escalabilidade
- Verificar sustentabilidade do modelo de crescimento

### Para Cluster 3 (Vendedores premium):
- Desenvolver seção exclusiva no marketplace para produtos premium
- Oferecer serviços especiais para este segmento (embalagem premium, entrega prioritária)
- Analisar estratégias para ampliar base de clientes mantendo posicionamento premium